In [ ]:
from datetime import datetime as dt
import csv, os, glob
from pprint import pprint

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import KFold
# from sklearn.preprocessing import label_binarize
from scipy import interp

import tensorflow.keras as k
from tensorflow.math import confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

# load tensorboard
%load_ext tensorboard

In [ ]:
def draw_CM(label, predicted):
    cm = confusion_matrix(label, predicted)
    sns.heatmap(cm, annot=True, fmt="d")
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

    # true : false rate
    true = 0
    false = 0
    for i, j in enumerate(label):
        if j != predicted[i]:
            false += 1
        else: true += 1

    classification_report = metrics.classification_report(label, predicted)
    multilabel_to_binary_matrics = metrics.multilabel_confusion_matrix(label, predicted)

    return plt.show(), print('true rate: ', true), print('false rate: ', false), print(), print('='*10, 'classification_report: ', '\n', classification_report), print('='*10, 'multilabel_to_binary_matrics by class_num: ','\n','[[TN / FP] [FN / TP]]','\n', multilabel_to_binary_matrics)
    
def draw_ROC_AUC(x, y, category_names):
    n_classes = len(category_names)

    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y[:, i], x[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    fpr["micro"], tpr["micro"], _ = roc_curve(y.ravel(), x.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
            label=f'micro-average ROC curve (area = {roc_auc["micro"]:0.2f})', 
            color='deeppink', linestyle=':', linewidth=1)

    plt.plot(fpr["macro"], tpr["macro"],
            label=f'macro-average ROC curve (area = {roc_auc["macro"]:0.2f})',
            color='navy', linestyle=':', linewidth=1)

    colors = (['purple', 'pink', 'red', 'green', 'yellow', 'cyan', 'magenta', 'blue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=1, label=f'Class {i} ROC curve (area = {roc_auc[i]:0.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.xlim([-.01, 1.0])
    plt.ylim([0.0, 1.01])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC & AUC')
    plt.legend(loc="lower right")

    return plt.show()

In [ ]:
# retrain
model_name = "model/model_"
model = k.models.load_model(model_name)

In [ ]:
path = '/home/z/data/label/'
for file in glob.glob(path + "*.csv"):
    df = pd.read_csv(file)
    df = df[['device_field03', 'illuminance_onoff', 'device_data_reg_dtm']]
    light_data = df['device_field03']
    label_data = df['illuminance_onoff']
    time_data = df['device_data_reg_dtm']
    start = 230
    end = 245

    print(light_data.shape)
    print(label_data.shape)
    print(df)

In [ ]:
def to_sequences(seq_size, t1, t2):
    x = []
    y = []
    for i in range(len(t1)-seq_size):
        ta1 = t1[i:(i+seq_size)]
        ta2 = t2[i-1+seq_size]
        # ta1 = [[x] for x in ta1]
        x.append(ta1)
        y.append(ta2)
        
    return np.array(x), np.array(y) 

timesteps = 6
x_train, y_train = to_sequences(timesteps,train_data, train_label)
x_test, y_test = to_sequences(timesteps,test_data, test_label)

In [ ]:
print(x_test[start:end])
print(y_test[start:end])

In [ ]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_train[100:105])
print(y_train[100:105])
print(x_test[100:105])
print(y_test[100:105])

In [ ]:
# # plot
# fig = px.line(df, x='device_data_reg_dtm', y=['device_field03', 'illuminance_onoff'])
# fig.update_xaxes(rangeslider_visible=True)
# fig.show(renderer='browser')

In [ ]:
loss, acc = model.evaluate(x_test, y_test, verbose=2)
# print(f'test_loss: {loss} test_accuracy: {acc}')

In [ ]:
predict = model.predict(x_test)
predicted = np.argmax(predict, axis=1)
y1 = np.argmax(y_test, axis=1)

In [ ]:
print(y1[:100])

In [ ]:
# CM
draw_CM(y1, predicted)

In [ ]:
# ROC, AUC
draw_ROC_AUC(predict, y_test, classes)

In [ ]:
# # launch tensorboard @ localhost:6006
# %tensorboard --logdir logs/ --host localhost --port 6006

In [ ]:
# df.to_csv("data.csv", index=False)